### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2019-01-02 17:34:32--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com... 162.125.70.1
Connecting to www.dropbox.com|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2019-01-02 17:34:33--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucabccd4f3c23b2fb8afdac17e4b.dl.dropboxusercontent.com/cd/0/get/AYqc9EuFVop62pQIjRWLUE6dyXnmM38FeMCKU2uj1y1trNW3LIeodHFJc5GMQSnxaK4pIJiAFK3ps5m0V-cxk0Myv9TXQLKJGeMV_1PD1u1ZkqE4jrwVO-CW_Z6WZemX97xrdZpmzKqEKgQFCwJGnRbYCCmoa37mzhj-OBfJPCght-DsRvu--N5b_SY7-iP7eWc/file?dl=1 [following]
--2019-01-02 17:34:33--  https://ucabccd4f3c23b2fb8afdac17e4b.dl.dropboxusercontent.com/cd/0/get/AYqc9EuFVop62pQIjRWLUE6dyXnmM38FeMCKU2uj1y1trNW3LIeodHFJc5GMQSnxaK4pIJiAFK3ps5m0V-c

,author,day,id,link,month,summary,tag,title,year
11365,"[{'name': 'Mengdi Wang'}, {'name': 'Yichen Che...",12,1511.03760v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Consider convex optimization problems subject ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Random Multi-Constraint Projection: Stochastic...,2015
24763,"[{'name': 'Sung Ju Hwang'}, {'name': 'Leonid S...",18,1411.5879v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,We propose a method that learns a discriminati...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",A Unified Semantic Embedding: Relating Taxonom...,2014
39055,"[{'name': 'Hamza Bendaoudi'}, {'name': 'Farida...",6,1612.09524v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,This paper presents a memory efficient archite...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Memory Efficient Multi-Scale Line Detector Arc...,2016
2254,"[{'name': 'Lin Feng'}, {'name': 'Shuliang Xu'}...",30,1710.10824v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Extreme learning machine (ELM) is a new single...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Rough extreme learning machine: a new classifi...,2017
31522,"[{'name': 'Weipeng Xu'}, {'name': 'Avishek Cha...",15,1803.05959v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,We propose the first real-time approach for th...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Mo2Cap2: Real-time Mobile 3D Motion Capture wi...,2018


In [21]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [22]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokinizer = WordPunctTokenizer()
lines = tokinizer.tokenize_sents([line.lower() for line in lines])
lines = list(map(lambda line: ' '.join(line), lines))

In [23]:
lines[:3]

['dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question . our single model outperforms the first place winner on the vqa 1 . 0 dataset , performs within margin to the current state - of - the - art ensemble model . we also experiment with replacing attention mechanisms in other state - of - the - art models with our implementation and show increased accuracy . in both cases , our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the vqa dataset .',
 'sequential short - text classification with recurrent and convolutional neural networks ; recent approaches based on artificial neural networks ( ann

In [24]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [19]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

In [31]:
def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        line = [UNK] * (n - 1) + line.split() + [EOS]
        for i in range(n - 1, len(line)):
            counts[tuple(line[i - (n - 1): i])][line[i]] += 1
    
    return counts

In [34]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [68]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        self.counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
#         self.probs = defaultdict(Counter)
        self.probs = self.counts.copy()
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in self.counts.keys():
            norm_coef = sum(self.probs[prefix][word] for word in self.probs[prefix].keys())
            for word in self.probs[prefix].keys():
                self.probs[prefix][word] /= norm_coef
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        return self.probs[self._preproc_prefix(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)
    
    def _preproc_prefix(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a tuple of last tokens (n - 1) tokens in prefix
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return tuple(prefix)

Let's test it!

In [69]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [70]:
%%time
lm = NGramLanguageModel(lines, n=3)

CPU times: user 21.4 s, sys: 544 ms, total: 21.9 s
Wall time: 22.1 s


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [71]:
sorted([(k, v) for k, v in lm.get_possible_next_tokens("neural").items()], key=lambda pair: -pair[1])

[('network', 0.12831858407079647),
 ('networks', 0.12389380530973451),
 ('machine', 0.11504424778761062),
 ('-', 0.02654867256637168),
 ('semantic', 0.017699115044247787),
 ('architectures', 0.017699115044247787),
 ('multi', 0.017699115044247787),
 ('text', 0.01327433628318584),
 ('variational', 0.01327433628318584),
 ('program', 0.01327433628318584),
 ('models', 0.008849557522123894),
 ('question', 0.008849557522123894),
 ('architecture', 0.008849557522123894),
 ('symbolic', 0.008849557522123894),
 ('style', 0.008849557522123894),
 ('discourse', 0.008849557522123894),
 ('system', 0.008849557522123894),
 ('turing', 0.008849557522123894),
 ('reranking', 0.008849557522123894),
 ('net', 0.008849557522123894),
 ('word', 0.008849557522123894),
 ('paraphrase', 0.008849557522123894),
 ('lattice', 0.008849557522123894),
 ('enquirer', 0.004424778761061947),
 ('associative', 0.004424778761061947),
 ('module', 0.004424778761061947),
 ('photo', 0.004424778761061947),
 ('responding', 0.004424778761

In [72]:
sorted([(k, v) for k, v in lm.get_possible_next_tokens("bridging the").items()], key=lambda pair: -pair[1])

[('gap', 0.7435897435897436),
 ('synthetic', 0.02564102564102564),
 ('language', 0.02564102564102564),
 ('subsymbolic', 0.02564102564102564),
 ('relatively', 0.02564102564102564),
 ('difference', 0.02564102564102564),
 ('rbm', 0.02564102564102564),
 ('disambiguated', 0.02564102564102564),
 ('information', 0.02564102564102564),
 ('insights', 0.02564102564102564),
 ('topological', 0.02564102564102564)]

In [74]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    token_probs = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        probs = np.zeros(len(token_probs))
        probs[np.argmax(list(token_probs.values()))] = 1
    else:
        degree = 1 / (temperature + 1e-10)
        probs = np.array([v**degree for v in token_probs.values()])
        probs = probs / sum(probs)
    return np.random.choice(list(token_probs.keys()), p=probs)

In [75]:
from collections import Counter

test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [76]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial intelligence is essentially an algorithm is an improved entropy number bound by doerr and doerr ( gecco 2015 ). the new digital synaptic neural substrate ' ( ogs ) regularization is important for improving end of the objective of each domain , we also release object bounding box coordinates and the voice structure of the image processing ; we present an algorithm ( mnih et al .' s to 1970 ' s new assertion , or in a multilingual corpus used in image retrieval . in this paper , we complement our generalization is concerned with recovery from uncalibrated cameras


In [77]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two models are the best performing models . we show that the proposed method is based on the imagenet dataset . _EOS_


In [80]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between these two modalities generally leads to a wide range of problems that are used for the purpose of this paper , we propose a new model , we propose a new objective function . the proposed method is that the proposed method is also presented . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [81]:
' a'.split()

['a']

In [94]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    n = 0
    log_perplexity = 0 
    for line in lines:
        prefix = ''
        for token in line.split() + [EOS]:
            log_perplexity += max(np.log(lm.get_next_token_prob(prefix, token)), min_logprob)
            prefix += ' ' + token
            n += 1
    return np.exp(-log_perplexity / n)

In [95]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/Users/maxim/code/git/kaggle/kaggle_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [96]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

/Users/maxim/code/git/kaggle/kaggle_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [97]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [98]:
# test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [99]:
from tqdm import tqdm

for n in tqdm((1, 2, 3)):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [103]:
from tqdm import tqdm

for n in tqdm((1, 2, 3)):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.01)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

 33%|███▎      | 1/3 [00:31<01:02, 31.48s/it]

N = 1, Perplexity = 977.93963


 67%|██████▋   | 2/3 [01:04<00:32, 32.09s/it]

N = 2, Perplexity = 290.88339


100%|██████████| 3/3 [01:48<00:00, 35.37s/it]

N = 3, Perplexity = 1573.42741


In [104]:
from tqdm import tqdm

for n in tqdm((1, 2, 3)):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.001)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  0%|          | 0/3 [00:00<?, ?it/s]/Users/maxim/code/git/kaggle/kaggle_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
 33%|███▎      | 1/3 [00:31<01:03, 31.87s/it]

N = 1, Perplexity = 1832.23547


 67%|██████▋   | 2/3 [01:04<00:32, 32.11s/it]

N = 2, Perplexity = 282.27785


100%|██████████| 3/3 [01:48<00:00, 35.67s/it]

N = 3, Perplexity = 1073.32808


In [115]:
lm = LaplaceLanguageModel(lines, n=3, delta=0.001)

In [116]:
sorted([(k, v) for k, v in lm.get_possible_next_tokens("bridging the").items()], key=lambda pair: -pair[1])

[('gap', 0.28499130315150206),
 ('insights', 0.009836774402767267),
 ('disambiguated', 0.009836774402767267),
 ('rbm', 0.009836774402767267),
 ('topological', 0.009836774402767267),
 ('difference', 0.009836774402767267),
 ('relatively', 0.009836774402767267),
 ('information', 0.009836774402767267),
 ('synthetic', 0.009836774402767267),
 ('subsymbolic', 0.009836774402767267),
 ('language', 0.009836774402767267),
 ('dmds', 9.826947455311959e-06),
 ('roundworms', 9.826947455311959e-06),
 ('recalibrates', 9.826947455311959e-06),
 ('typhoons', 9.826947455311959e-06),
 ('velopment', 9.826947455311959e-06),
 ('@', 9.826947455311959e-06),
 ('irl2', 9.826947455311959e-06),
 ('58', 9.826947455311959e-06),
 ('formalise', 9.826947455311959e-06),
 ('supervoxel', 9.826947455311959e-06),
 ('picture', 9.826947455311959e-06),
 ('multilevel', 9.826947455311959e-06),
 ('grobner', 9.826947455311959e-06),
 ('atomnet', 9.826947455311959e-06),
 ('horn', 9.826947455311959e-06),
 ('utilising', 9.82694745531195

In [117]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the source code , and the size of the proposed method produces state - of - the - art methods . _EOS_


In [105]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the outputs of the model to predict the next one . _EOS_


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=5.0):
        self.n = n
        
        self.counts = {i: count_ngrams(lines, i) for i in range(1, self.n + 1)}
        self.vocab = set(token for token_counts in self.counts[1].values() for token in token_counts)
        
        # compute token proabilities given counts
        self.probs = defaultdict(defaultdict(Counter))
#         self.probs = self.counts.copy()
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in self.counts.keys():
            for i in range(1, self.n + 1):
                token_counts = counts[i][prefix[self.n - i:]]
                total_count = sum(token_counts.values())# + delta * len(self.vocab)
                self.probs[i][prefix] = {
                    token: max(0, token_counts[token] - delta) / total_count
                    for token in token_counts
                }
            
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
            
            
            
            norm_coef = sum(self.probs[prefix][word] for word in self.probs[prefix].keys())
            for word in self.probs[prefix].keys():
                self.probs[prefix][word] /= norm_coef
        
    def get_possible_next_tokens(self, prefix):
        < YOUR CODE >
        
    def get_next_token_prob(self, prefix, next_token):
        <YOUR CODE>

In [97]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [68]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        self.counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
#         self.probs = defaultdict(Counter)
        self.probs = self.counts.copy()
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in self.counts.keys():
            norm_coef = sum(self.probs[prefix][word] for word in self.probs[prefix].keys())
            for word in self.probs[prefix].keys():
                self.probs[prefix][word] /= norm_coef
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        return self.probs[self._preproc_prefix(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)
    
    def _preproc_prefix(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a tuple of last tokens (n - 1) tokens in prefix
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return tuple(prefix)

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))